# Imports

In [48]:
import pandas as pd
import plotly.express as px

# Load Data

In [49]:
df = pd.read_csv("../datasets/BreadBasket_DMS_DT.csv")

# Structure and unique values of our dataset

In [50]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21293 entries, 0 to 21292
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Transaction  21293 non-null  int64 
 1   Item         21293 non-null  object
 2   DateTime     21293 non-null  object
dtypes: int64(1), object(2)
memory usage: 499.2+ KB


,Transaction
count,21293.000000
mean,4951.990889
std,2787.758400
min,1.000000
25%,2548.000000
50%,5067.000000
75%,7329.000000
max,9684.000000


In [51]:
df.head(10)

,Transaction,Item,DateTime
0,1,Bread,2016-10-30 09:58:11
1,2,Scandinavian,2016-10-30 10:05:34
2,2,Scandinavian,2016-10-30 10:05:34
3,3,Hot chocolate,2016-10-30 10:07:57
4,3,Jam,2016-10-30 10:07:57
5,3,Cookies,2016-10-30 10:07:57
6,4,Muffin,2016-10-30 10:08:41
7,5,Coffee,2016-10-30 10:13:03
8,5,Pastry,2016-10-30 10:13:03
9,5,Bread,2016-10-30 10:13:03


In [52]:
df.tail(10)

,Transaction,Item,DateTime
21283,9681,Tea,2017-04-09 14:30:09
21284,9681,Spanish Brunch,2017-04-09 14:30:09
21285,9681,Christmas common,2017-04-09 14:30:09
21286,9682,Muffin,2017-04-09 14:32:58
21287,9682,Tacos/Fajita,2017-04-09 14:32:58
21288,9682,Coffee,2017-04-09 14:32:58
21289,9682,Tea,2017-04-09 14:32:58
21290,9683,Coffee,2017-04-09 14:57:06
21291,9683,Pastry,2017-04-09 14:57:06
21292,9684,Smoothies,2017-04-09 15:04:24


In [53]:
uniqueItems = df["Item"].unique()
print("There are " + str(len(uniqueItems)) + " unique items:")
print(uniqueItems)

There are 95 unique items:
['Bread' 'Scandinavian' 'Hot chocolate' 'Jam' 'Cookies' 'Muffin' 'Coffee'
 'Pastry' 'Medialuna' 'Tea' 'NONE' 'Tartine' 'Basket' 'Mineral water'
 'Farm House' 'Fudge' 'Juice' "Ella's Kitchen Pouches" 'Victorian Sponge'
 'Frittata' 'Hearty & Seasonal' 'Soup' 'Pick and Mix Bowls' 'Smoothies'
 'Cake' 'Mighty Protein' 'Chicken sand' 'Coke' 'My-5 Fruit Shoot'
 'Focaccia' 'Sandwich' 'Alfajores' 'Eggs' 'Brownie' 'Dulce de Leche'
 'Honey' 'The BART' 'Granola' 'Fairy Doors' 'Empanadas' 'Keeping It Local'
 'Art Tray' 'Bowl Nic Pitt' 'Bread Pudding' 'Adjustment' 'Truffles'
 'Chimichurri Oil' 'Bacon' 'Spread' 'Kids biscuit' 'Siblings'
 'Caramel bites' 'Jammie Dodgers' 'Tiffin' 'Olum & polenta' 'Polenta'
 'The Nomad' 'Hack the stack' 'Bakewell' 'Lemon and coconut' 'Toast'
 'Scone' 'Crepes' 'Vegan mincepie' 'Bare Popcorn' 'Muesli' 'Crisps'
 'Pintxos' 'Gingerbread syrup' 'Panatone' 'Brioche and salami'
 'Afternoon with the baker' 'Salad' 'Chicken Stew' 'Spanish Brunch'
 'Ras

# Drop Null/None values
We drop columns with "NONE" value as we have seen in the unique items list

In [54]:
df = df[df.Item != "NONE"]

# Pie Chart for the 10 most common items

We get the 10 most common items with their counts in a Pandas Series, and label the rest as "Others".

In [55]:
mostPopularItems = df.Item.value_counts()[:10]
otherItemsCount = df.Item.count() - mostPopularItems.sum()
allItemsCount = mostPopularItems.append(pd.Series([otherItemsCount], index = ["Others"]))
allItemsCount

Coffee           5471
Bread            3325
Tea              1435
Cake             1025
Pastry            856
Sandwich          771
Medialuna         616
Hot chocolate     590
Cookies           540
Brownie           379
Others           5499
dtype: int64

In [56]:
fig = px.pie(allItemsCount, values=0, names=allItemsCount.index, title="Most Popular Items on our Bakery", color_discrete_sequence = px.colors.sequential.Mint_r)
fig['layout'].update(width=750, height=750, autosize=False)
fig.show()

# Number of Sales by Timeframe (Day, Month, Hour, etc.)

We show the count in a histogram with as many bins as unique months there are, to plot it divided by months chronologically 

In [57]:
df["DateTime"] = pd.to_datetime(df["DateTime"], utc=True)
monthsCount = df.DateTime.dt.month_name().unique()
fig = px.histogram(df, x = "DateTime", nbins = len(monthsCount), title = "Sales per month in chronological order", color_discrete_sequence = px.colors.sequential.Mint_r)
fig.update_layout(bargap=0.2)
fig.show()

We show the count in a histogram divided by months to see the most productive month

In [58]:
countByMonth = df.groupby(df.DateTime.dt.month_name())['Item'].count().sort_values(ascending=False)
fig = px.bar(countByMonth, title = "Most Productive Month", color=countByMonth, color_continuous_scale=px.colors.sequential.Mint)
fig.show()

We repeat the process per day and timeframe (Morning, Noon, Eve)

In [59]:
countByDay = df.groupby(df.DateTime.dt.day_name())['Item'].count().sort_values(ascending=False)
fig = px.bar(countByDay, title = "Most Productive Day", color=countByDay, color_continuous_scale=px.colors.sequential.Mint)
fig.show()

In [60]:
countbyHour = pd.DataFrame({"Time":range(1, 25)})
countbyHour["Count"] = df.groupby(df.DateTime.dt.hour.sort_values(ascending=True))["Item"].count()
countbyHour["Count"] = countbyHour["Count"].fillna(0)
fig = px.bar(countbyHour, x = "Time", y = "Count", title = "Sales per Hour of the Day", color=countbyHour["Count"], color_continuous_scale=px.colors.sequential.Mint)
fig.show()

In [61]:
hours = [0,12,18,24]
sessions = ["Morning","Afternoon","Evening"]
countbyHour["Session"] = pd.cut(countbyHour["Time"], bins=hours, labels=sessions, include_lowest=True)
countbyHour = countbyHour.groupby(countbyHour.Session)["Count"].sum().reset_index()
fig = px.pie(countbyHour, names = "Session", values = "Count", title = "Peak Selling Hours", color_discrete_sequence = px.colors.sequential.Mint_r)
fig['layout'].update(width=500, height=500, autosize=False)
fig.show()

# Most common products by timeframe

## By day

We explore if each day has a similar distribution of most common bought products.

In [118]:
import plotly.graph_objects as go
import calendar
days = list(calendar.day_name)

domains = [
    {'x': [0.0, 0.33], 'y': [0.0, 0.33]},
    {'x': [0.33, 0.66], 'y': [0.0, 0.33]},
    {'x': [0.66, 1.0], 'y': [0.0, 0.33]},
    {'x': [0.0, 0.33], 'y': [0.33, 0.66]},
    {'x': [0.33, 0.66], 'y': [0.33, 0.66]},
    {'x': [0.66, 1.0], 'y': [0.33, 0.66]},
    {'x': [0.33, 0.66], 'y': [0.66, 1.0]},
    {'x': [0.0, 0.33], 'y': [0.66, 1.0]},
    {'x': [0.66, 1.0], 'y': [0.66, 1.0]}
]

domainCount = 0
traces = []

for day in days:
    mostPopularItems = df[df.DateTime.dt.day_name() == day].Item.value_counts()[:10]
    otherItemsCount = df[df.DateTime.dt.day_name() == day].Item.count() - mostPopularItems.sum()
    allItemsCount = mostPopularItems.append(pd.Series([otherItemsCount], index = ["Others"]))
    trace = go.Pie(labels = allItemsCount.index,
                   values = allItemsCount,
                   domain = domains[domainCount],
                   hoverinfo = 'label+percent+name',
                   title = "Common Products on " + day)
    traces.append(trace)
    domainCount += 1


mostPopularItems = df.Item.value_counts()[:10]
otherItemsCount = df.Item.count() - mostPopularItems.sum()
allItemsCount = mostPopularItems.append(pd.Series([otherItemsCount], index = ["Others"]))
trace = go.Pie(labels = allItemsCount.index,
                values = allItemsCount,
                domain = domains[domainCount],
                hoverinfo = 'label+percent+name',
                title = "Common Products")
traces.append(trace)

layout = go.Layout(height = 1400,
                   width = 1400,
                   autosize = False,
                   title = 'Most popular items per day of the week')
fig = go.Figure(data = traces, layout = layout)
fig.show()